In [1]:
import cv2
from segment_anything import build_sam, SamAutomaticMaskGenerator
from PIL import Image, ImageDraw
import clip
import torch
import numpy as np

In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Download the model weights to load them here
sam = build_sam(checkpoint="sam_vit_h_4b8939.pth")
# sam = sam.to(device)
# mask_generator = SamAutomaticMaskGenerator(sam)

In [3]:
def add_RGB_segment(result, segmentation_mask, annotation):
    result[segmentation_mask] = annotation[segmentation_mask]

In [4]:
import os
import glob

img_dir = 'isaid_segm/val/images/images'
image_path_list = glob.glob(os.path.join(img_dir, '*.png'))

In [5]:
def update_patch(mask_generator, image, annotation, result, device):
    masks = mask_generator.generate(image)

    for mask in masks:
        mask["segmentation"] = torch.tensor(mask["segmentation"]).to(device)
        add_RGB_segment(result, mask["segmentation"], annotation)

def run(image_path, segpath, mask_generator, device):
    image = Image.open(image_path)
    image = np.array(image)
    if len(image.shape) == 2:
        image = np.stack([image] * 3, axis=2)

    file_name = os.path.basename(image_path)
    print(file_name)
    annotation = Image.open(os.path.join(segpath, file_name))
    annotation = np.array(annotation)
    annotation = torch.tensor(annotation).to(device)
    # display(annotation)

    result = torch.zeros_like(annotation).to(device)
    
    step = 800
    for h_start in range(0, image.shape[0], step):
        for w_start in range(0, image.shape[1], step):
            h_end = np.min([h_start+step, image.shape[0]])
            w_end = np.min([w_start+step, image.shape[1]])
            print(h_start, h_end, w_start, w_end)
            update_patch(mask_generator, image[h_start:h_end, w_start:w_end], 
                         annotation[h_start:h_end, w_start:w_end], result[h_start:h_end, w_start:w_end], device)

    if not os.path.exists(segpath+'_fixed'):
        os.mkdir(segpath+'_fixed')
    Image.fromarray(result.cpu().numpy()).save(os.path.join(segpath+'_fixed', file_name))

def run_list(image_path_list, segpath, mask_generator, device):
    for image_path in image_path_list:
        run(image_path, segpath, mask_generator, device)


In [6]:
from torch import multiprocessing

segpath = "farsegResultWithThreshold=0.5"

device_id=[0,1,2,3,4,5,6,7]
mask_generator_list=[]
for device in device_id:
    mask_generator_list.append(SamAutomaticMaskGenerator(sam.to(device)))

new_image_path_list = np.array_split(np.array(image_path_list), 8)

multiprocessing.set_start_method('spawn')
workers = [multiprocessing.Process(
                    target=run_list,
                    args=(new_image_path_list[i], segpath, mask_generator_list[i], torch.device("cuda:"+str(i))))
                for i in range(8)]


for worker in workers:
    worker.start()


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/data/zxl/miniconda3/envs/CLIP-SAM/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/data/zxl/miniconda3/envs/CLIP-SAM/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'run_list' on <module '__main__' (built-in)>


KeyboardInterrupt: 